# Types and Multiple Dispatch

Pearl Li

December 18, 2017

### Outline

1. Types
2. Multiple Dispatch
3. Writing Julian Code

## Types

A **data type** is a classification identifying the kind of data you have. An object’s type determines the possible values it can take on, which operations and functions can be applied to it, and how the computer stores it.

Examples:

- Numeric types: `Int64`, `Float64`
- String types: `String`, `SubString`
- `Bool`
- `Array`

Names of types are written in UpperCamelCase.

A **concrete instance** (also an object or a value) of a type `T` is a piece of data in memory that has type `T`.

Variables are not data, but are simply names that point/refer to a specific piece of data. The underlying data that a variable refers to has a specific type.

In [ ]:
# What is the type of 10?
typeof(10)

In [ ]:
# Is 10 an Int64?
isa(10, Int64)

In [ ]:
# What is the type of the elements of an array?
X = [1.0, 2.0, 3.0]
eltype(X)

### Composite Types

A **composite type** is a collection of named fields that can be treated as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`, constrains a field to contain values of a certain type. This is optional for any field.

**Mutable** composite types are defined using the keywords `mutable struct`.

In [1]:
# Type definition
mutable struct Parameter
    value::Float64
    transformation::Function # Function is a type!
    tex_label::String
    description::String
end

When a type with $n$ fields is defined, a constructor (function that creates an instance of that type) that takes $n$ ordered arguments is automatically created. Additional constructors can be defined for convenience.

In [ ]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\beta", "Discount rate")

In [ ]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

In [ ]:
# Find the fields of an instance of a composite type
fieldnames(α)

In [ ]:
# Access a particular field using .
α.value

In [ ]:
# Fields in a mutable composite type are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

### Subtyping

Types are hierarchically related to each other. All are subtypes of the `Any` type.

There are two main kinds of types in Julia:

1. Concrete types: familiar types that you can create instances of, like `Int64` or `Float64`.
2. Abstract types: nodes in a type graph that serve to group similar kinds of objects. Abstract types cannot be instantiated and do not have explicitly declared fields. For example, `Integer` or `Number`.

In [ ]:
# Define an abstract type
abstract type Model end

In [ ]:
# Define concrete subtypes of that abstract type
mutable struct VAR <: Model
    n_lags::Int64
    variables::Vector{Symbol}
    data::Matrix{Float64}
end

In [ ]:
# Check subtyping relation
VAR <: Model

In [ ]:
# Instances of the VAR type are also instances of the Model type
model = VAR(1, [:gdp, :inflation])
isa(model, Model)

In [ ]:
# Why does this throw an error?
3 <: Number

### Parameterized Types

**Parameterized types** are data types that are defined to handle values identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [2]:
# Defining a parametric point
struct Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

This single declaration defines an unlimited number of new types: `Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [3]:
Duple(3, -15)

Duple{Int64}(3, -15)

In [ ]:
Duple("Broadway", "42nd St")

In [ ]:
# What happens here?
Duple(1.5, 3)

Notice that we defined `Duple` without the `mutable` keyword. We can still access the fields of `Duple` instances as before, but now attempts to modify the fields will cause an error to be thrown.

In [6]:
pt = Duple(3, -15)
pt.x

3

In [5]:
pt.x = 0

LoadError: [91mtype Duple is immutable[39m

Each particular parameterization of a parametric type is a type, and moreover a subtype of the parametric type:

In [12]:
typeof(Duple{Int})

DataType

In [15]:
Duple{Int} <: Duple

true

We can also restrict the type parameter `T`:

In [ ]:
# T can be any subtype of Number, but nothing else
type PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [ ]:
PlanarCoordinate("4th Ave", "14th St")

### Type Aliases

**Type aliases** can be used to simplify references to existing types using regular assignment. For example, `Vector{T}` and `Matrix{T}` are type aliases for `Array{T, 1}` and `Array{T, 2}` respectively:

In [7]:
Vector{T} = Array{T, 1}
Matrix{T} = Array{T, 2}

Array{T,2} where T

In [9]:
x = [1, 2, 3]
isa(x, Array{Int, 1})

true

In [10]:
isa(x, Vector{Int})

true

In [11]:
Vector{Int} == Array{Int, 1}

true

## Exercise

In [ ]:
using Distributions

The [Distributions](https://juliastats.github.io/Distributions.jl/latest/index.html) package implements sampling, moments, PDFs, and more for many distributions. `Distribution` is an abstract parameterized type, and `ContinuousUnivariateDistribution` and `ContinuousMultivariateDistribution` are two specific (aliases for) abstract subtypes of `Distribution`.

```
abstract Distribution{F<:VariateForm,S<:ValueSupport} <: Sampleable{F,S}
const ContinuousUnivariateDistribution   = Distribution{Univariate,   Continuous}
const ContinuousMultivariateDistribution = Distribution{Multivariate, Continuous}
```

Implement the [triangular distribution](https://en.wikipedia.org/wiki/Triangular_distribution) ("a continuous probability distribution with lower limit $a$, upper limit $b$ and mode $c$, where $a < b$ and $a \leq c \leq b$") as an immutable composite type `Triangular`, which is a subtype of `ContinuousUnivariateDistribution`.

In [ ]:
struct Triangular ...

In addition to the default constructor, implement a second `Triangular` constructor which only takes in the lower and upper limits as arguments, setting the mode to $\frac{a+b}{2}$.

In [ ]:
function Triangular(a, b)
    ...
end

### Why Use Types?

You can write all your code without thinking about types at all. If you do this, however, you’ll be missing out on some of the biggest benefits of using Julia.

If you understand types, you can:

- Write faster code
- Write expressive, clear, and well-structured programs (keep this in mind when we talk about functions)
- Reason more clearly about how your code works

From the QuantEcon lecture on [Types, Methods, and Dispatch](https://lectures.quantecon.org/jl/types_methods.html#motivation):

> At our respective homes we both have drawers full of fishing gear. Of course we have drawers full of other things too, like kitchen utensils, or clothes. Are these drawers really necessary? Perhaps not, but who wants to search the whole house for their fishing reel when the fish are biting? Certainly not us. Just as it’s convenient to store household objects in drawers, **it’s also convenient to organize the objects in your program into designated “containers”**.

Even if you only use built-in functions and types, your code still takes advantage of Julia’s type system. That’s why it’s important to understand what types are and how to use them.

In [ ]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Float64)  
    sum = 0
    for i in 1:n
        sum += sin(3.4)
    end
    return sum
end  

function sumofsins_stable(n::Float64)  
    sum = 0.0
    for i in 1:n
        sum += sin(3.4)
    end
    return sum
end

# Compile and run
sumofsins_unstable(1e5)
sumofsins_stable(1e5)

In [ ]:
@time sumofsins_unstable(1e5)

In [ ]:
@time sumofsins_stable(1e5)

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type `Float64` throughout; therefore, it saves time and memory. On the other hand, in `sumofsins_unstable`, the compiler must check the type of `sum` at each iteration of the loop. Let's look at the LLVM [intermediate representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

### Exercise

(Taken from QuantEcon's [Vectors, Arrays, and Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lecture.)

Write a function `solve_discrete_lyapunov` that solves the discrete Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative procedure $$S_0 = \Sigma \Sigma'$$ $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a number of iterations. 
You can assume that your $A$ and $\Sigma$ matrices are of type `Matrix{Float64}` and the number of iterations is an `Integer`. Make sure to check your code for type stability!

## Multiple Dispatch

So far we have defined functions over argument lists of any type. Methods allow us to define functions “piecewise”. For any set of input arguments, we can define a **method**, a definition of one possible behavior for a function.

In [ ]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

In [ ]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

In [ ]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

In [ ]:
# See all methods for a given function
methods(print_type)

Julia uses **multiple dispatch** to decide which method of a function to execute when a function is applied. In particular, Julia compares the types of _all_ arguments to the signatures of the function’s methods in order to choose the applicable one, not just the first (hence "multiple").

In [ ]:
print_type(5)

In [ ]:
print_type("foo")

In [ ]:
# This throws an error because no method of print_type has been
# defined for this set of arguments
print_type([1, 2, 3])

How is multiple dispatch useful for economic research? Recall that we defined the type `VAR` earlier, and made it a subtype of our abstract type `Model`. Let's define another subtype of `Model`:

In [ ]:
# Define a general linear model
type GLM <: Model
    y_variables::Vector{Symbol}
    x_variables::Vector{Symbol}
    y_data::Matrix{Float64} # Nt * Ny
    x_data::Matrix{Float64} # Nt * Nx
end

Now we can use the same function name, `estimate`, to define different estimation behaviors for the different subtypes of `Model`:

In [ ]:
using Distributions

function estimate(model::GLM)
    # Estimate a general linear model using OLS
end

function estimate(model::VAR)
    # Estimate a VAR using maximum likelihood
end

function estimate(model::VAR, prior::Distribution)
    # Estimate a Bayesian VAR
end

In [ ]:
methods(estimate)

### Exercise

Implement the function `estimate(model::GLM)` using the given `GLM` type. That is, return a matrix of size $N_x \times N_y$ of coefficients estimated using OLS. You may find the `pinv` and `inv`, or `qr` functions helpful.

Test it on the following model:

In [ ]:
β = ones(2, 1)                     # Nx x Ny
x_data = rand(1000, 2)             # Nt x Nx
y_data = x_data*β + randn(1000, 1) # Nt x Ny
model = GLM([:y1], [:x1, :x2], y_data, x_data)
# β_hat = estimate(model)

## Writing Julian Code

As we've seen, you can use Julia just like you use MATLAB and get faster code. However, to write faster and _better_ code, attempt to write in a “Julian” manner:

- Define composite types as logically needed
- Write type-stable functions for best performance
- Take advantage of multiple dispatch to write code that looks like math
- Add methods to existing functions

### Just-in-Time Compilation

How is Julia so fast? Julia is just-in-time (JIT) compiled, which  means (according to [this StackExchange answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do), with emphasis mine):

> A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. **A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently.**

> This is in contrast to a traditional compiler that compiles all the code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how your code is compiled. This is why writing type-stable code makes such a difference in speed!